In [1]:
import nglview as ng
import mdtraj as md
import pandas as pd

In [4]:
!gmx_mpi grompp -f nvt-charmm.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr 

                      :-) GROMACS - gmx grompp, 2025.0 (-:

Executable:   /usr/local/gromacs/bin/gmx_mpi
Data prefix:  /usr/local/gromacs
Working dir:  /home/ubuntu/src/gromacs
Command line:
  gmx_mpi grompp -f nvt-charmm.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr

Ignoring obsolete mdp entry 'title'
Setting the LD random seed to -4493441

Generated 20503 of the 20503 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 1

Generated 17396 of the 20503 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Protein_chain_L'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

Setting gen_seed to -1099195953

In [ ]:
!mpirun -np 3 gmx_mpi mdrun -deffnm nvt

                      :-) GROMACS - gmx mdrun, 2025.0 (-:

Executable:   /usr/local/gromacs/bin/gmx_mpi
Data prefix:  /usr/local/gromacs
Working dir:  /home/ubuntu/src/gromacs
Command line:
  gmx_mpi mdrun -deffnm nvt


Back Off! I just backed up nvt.log to ./#nvt.log.2#
Reading file nvt.tpr, VERSION 2025.0 (single precision)
Changing nstlist from 10 to 80, rlist from 1.2 to 1.316

Using 3 MPI processes
Using 1 OpenMP thread per MPI process


Back Off! I just backed up nvt.trr to ./#nvt.trr.2#

Back Off! I just backed up nvt.edr to ./#nvt.edr.2#
starting mdrun 'COAGULATION FACTOR XA'
50000 steps,    100.0 ps.
